# Chapter 8.10 - Planetary Ephemeris

To design realistic interplanetary missions, we must be able to determine the state vector of the relevant planets at a given time, or over a given time interval. This allows calculation of, for instance, the phase angle between planets to determine the window of time to reach the target planet.

The data that gives the state vector are called _planetary ephemerides_. Due to the chaotic nature of the orbits of the planets, we can only accurately predict planetary state vectors for a relatively short time into the future, about 50--100 years. The chaotic nature of the orbits is due to the complex interaction of the orbits with the gravitational potential from the Sun and Jupiter, primarily.

Ephemerides are also available for more minor objects in the solar system, such as planetary moons and asteroids. The uncertainty over long time intervals of these objects is even higher than for planets, because more objects have the ability to change the orbit of the smaller bodies.

There are several ways to calculate ephemerides. We will discuss three of them here:

1. Using simplified formulas from JPL
2. Using more accurate equations with a purpose-built library
3. Using the online HORIZONS database

In all three cases, we rely on knowing the [_Julian Day_](https://en.wikipedia.org/wiki/Julian_day) of interest.

## Julian Day

The Julian Day is a concept invented by historians and astronomers to provide a continuous count of days since a particular starting point. Julian Day 0 is set to be the day starting at January 1, 4713 BCE at 12:00 PM (noon) in UTC. After that, days are counted as integers continuously until the present time. This makes it very easy to compare relative times of events and do arithmetic between days.

The Julian Day is a single day in the Julian Period, a period of time 7,980 years long. The Julian Period is defined as the product of three cycles in the Julian calendar:

1. The solar cycle (28 years)
2. The lunar cycle (19 years)
3. The indiction cycle (15 years)

A Julian Period starts when each of these cycles is on its first day, on the same day. Counting backwards, and assuming that the Julian calendar extends infinitely in both directions in time (called the [_proleptic Julian calendar_](https://en.wikipedia.org/wiki/Proleptic_Julian_calendar)), the previous time that this occurred was in the year 4713 BCE.

### The Solar Cycle

The Julian calendar is the calendar imposed by Julius Cesar during the Roman Republic. It had a leap year every 4th year, with no exceptions. This gave the average length of a year as 365.25 days, keeping the days of the year in sync with the length of the solar year. Since there are 7 days in a week, it takes 28 years for a leap day to happen on every day of the week, giving the length of the [solar cycle](https://en.wikipedia.org/wiki/Solar_cycle_(calendar)).

### The Lunar Cycle

As we know, the moon goes through phases as it orbits around the earth. The [lunar, or _metonic_, cycle](https://en.wikipedia.org/wiki/Metonic_cycle) of 19 years is approximately the time for the lunar phase to repeat on the same day of the year.

### The Indiction Cycle

An [indiction](https://en.wikipedia.org/wiki/Indiction) is the periodic reassessment of property for a land tax, used in the Middle Ages. At that time, the reassessment occurred every 15 years.

### Calculating Julian Days

The Julian calendar was introduced in 46 BC by Julius Cesar, with leap years occuring every four years. This gives the average length of a year as 365.25 days, which is slightly longer than the [solar year](https://en.wikipedia.org/wiki/Tropical_year) of 365.2422 days. Therefore, over time, the day on which the equinox will occur shifts.

In 1582, Pope Gregory XIII imposed a modified version of the Julian calendar. The Gregorian calendar, which is still in use today, corrects the average length of the year to 365.2425 days by skipping leap years under certain conditions:

> The Gregorian leap year rule is: Every year that is exactly divisible by four is a leap year, except for years that are exactly divisible by 100, but these centurial years are leap years if they are exactly divisible by 400. For example, the years 1700, 1800, and 1900 are not leap years, but the year 2000 is. [Source](https://web.archive.org/web/20190919024611/https://aa.usno.navy.mil/faq/docs/calendars.php)

Since the Julian Period was introduced in 1583, it used the Julian calendar rather than the Gregorian calendar, which wouldn't be adopted everywhere until the mid-1700s. This means that we need some formula to convert a given Gregorian date into a Julian Day.

A Gregorian calendar date must have 3 integer parts to calculate the Julian Day Number ($\mathrm{JDN}$):

1. The month number, $M$, varying from 1-12.
2. The day of the month, $D$, varying from 1-31 depending on the month.
3. The year, $Y$, using [astronomical year numbering](https://en.wikipedia.org/wiki/Astronomical_year_numbering). Astronomical year numbering is the same as the Gregorian year for all after 1 AD. However, the Gregorian calendar does not include a Year 0, whereas the astronomical year numbering does. Therefore:

   * AD 2 = Astronomical Year 2 
   * AD 1 = Astronomical Year 1
   * 1 BC = Astronomical Year 0
   * 2 BC = Astronomical Year -1
   * 3 BC = Astronomical Year -2

To simplify the calculation of the Julian Day Number, we will define the following constants:

$$\begin{aligned}A &= \mathrm{INT}\left(\frac{M - 14}{12}\right)\\B &= 1461\left(Y + 4800 + A\right)\\C &= 367\left(M - 2 - 12A\right)\\E &= \mathrm{INT}\left(\frac{Y + 4900 + A}{100}\right)\end{aligned}$$

where $\mathrm{INT}$ indicates that integer division should be done. This means that the remainder of the division is discarded, or in other words, the result is rounded towards zero. Then, $\mathrm{JDN}$ can be found by:

$$\mathrm{JDN} = \mathrm{INT}\left(\frac{B}{4}\right) + \mathrm{INT}\left(\frac{C}{12}\right) - \mathrm{INT}\left(\frac{3E}{4}\right) + D - 32075$$

Sample implementations of this algorithm can be found in many places online, and for many programming languages. See:

* <https://web.archive.org/web/20140902005638/http://mysite.verizon.net/aesir_research/date/jdimp.htm>
* <https://archive.org/details/131123ExplanatorySupplementAstronomicalAlmanac/page/n315/mode/2up>, Page 604
* <https://web.archive.org/web/20201207022832/https://craftofcoding.files.wordpress.com/2013/07/cs_langjuliandates.pdf>
* <https://dl.acm.org/doi/10.1145/364096.364097>

One possible Python implementation is also included here.

In [1]:
def gregorian_to_julian_day_number(month, day, year):
    """Convert the given proleptic Gregorian date to the equivalent Julian Day Number."""
    if month < 1 or month > 12:
        raise ValueError("month must be between 1 and 12, inclusive")
    if day < 1 or day > 31:
        raise ValueError("day must be between 1 and 31, inclusive")
    A = int((month - 14) / 12)
    B = 1461 * (year + 4800 + A)
    C = 367 * (month - 2 - 12 * A)
    E = int((year + 4900 + A) / 100)
    JDN = int(B / 4) + int(C / 12) - int(3 * E / 4) + day - 32075
    return JDN

from datetime import date
today = date.today()
print(gregorian_to_julian_day_number(today.month, today.day, today.year))

2459191


```{note}
**Note**: In Python 3, integer division (with two slashes `//`) is also known as _floor division_, meaning the result is rounded towards negative infinity. However, the `int()` built-in function rounds floating point numbers towards zero, so in the function above, we use floating point division and then convert to an integer.
```

We can confirm that for the Julian Day beginning at 12:00 PM (noon) on December 7, 2020, the Julian Day Number is 2,459,191, for instance, with the NASA calculator at <https://core2.gsfc.nasa.gov/time/julian.html>.

## Julian Dates

Related to the Julian Day, the _Julian Date_ ($\mathrm{JDT}$) is a decimal number that includes the Julian Day number in the whole part, and the fraction of the day towards the next Julian Day Number in the decimal. Remember that Julian Days start at 12:00 PM (noon) UTC, so 6:00 PM UTC would be JDN + 0.25 and 6:00 AM UTC would be the JDN of the previous Gregorian date + 0.75. In Python, we can implement this as follows.

In [2]:
def gregorian_to_julian_date(dt):
    """Convert a Gregorian date to a Julian Date."""
    JDN = gregorian_to_julian_day_number(dt.month, dt.day, dt.year)
    JDT = JDN + (dt.hour - 12) / 24 + dt.minute / 1_440 + dt.second / 86_400 + dt.microsecond / 86_400_000_000
    return JDT

from datetime import datetime, timezone
now = datetime.now(tz=timezone.utc)
print(gregorian_to_julian_date(now))
morning = now.replace(hour=6, minute=0, second=0, microsecond=0)
print(gregorian_to_julian_date(morning))

2459191.3764835643
2459190.75


At the time the code was executed, it was approximately 8:00 PM UTC on December 7, 2020. Since this time is before noon on December 8, the Julian Day Number is still 2,459,191. The fraction of the day completed is about 0.36. If the time is replaced with 6:00 AM, then exactly 0.75 of the Julian Day has passed, but notice that the JDN is reduced by one, because 6:00 AM on December 7 is part of the Julian Day that began on December 6 at 12:00 PM (noon).

Note that we passed the `timezone.utc` to the `datetime.now()` call. This assigns the UTC timezone to `now`, and automatically uses the computer's clock to convert from your local time to the current UTC time.

MATLAB has a function called [`juliandate`](https://www.mathworks.com/help/matlab/ref/datetime.juliandate.html), which takes a [`datetime`](https://www.mathworks.com/help/matlab/ref/datetime.html) instance and does the conversion to the Julian Date automatically. Note that you should specify the local time zone if you use `datetime('now')` to ensure that the correct Julian Day Number is returned.

```matlab
>> format longG
>> t1 = datetime('now');
>> t1.TimeZone = 'America/New_York';
>> t1

t1 = 

  datetime

   06-Dec-2020 22:25:28

>> juliandate(t1)

ans =

          2459190.64268566
```

Note that we assigned the `'America/New_York'` time zone to this `datetime`, such that `juliandate()` internally converted the time to UTC before converting to the Julian Date. If no `TimeZone` is assigned, `juliandate()` will assume that the `datetime` is in UTC, which may not be correct. For instance, this was run immediately after the previous code sample, but shows a different Julian Date:

```matlab
>> juliandate(datetime('now'))

ans =

          2459190.43363142
```

MATLAB also allows the reverse conversion, by using `datetime()` with the Julian Date:

```matlab
>> jd1 = juliandate(t1);
>> datetime(jd1,'ConvertFrom','juliandate')

ans = 

  datetime

   07-Dec-2020 03:25:28
```

Note that the default time zone is UTC, to convert to a different time zone, the `'TimeZone'` argument to `datetime()` must be supplied:

```matlab
>> datetime(jd1,'ConvertFrom','juliandate','TimeZone','America/New_York')

ans = 

  datetime

   06-Dec-2020 22:25:28
```

In [3]:
from datetime import timedelta, time

def julian_day_number_to_gregorian(jdn):
    """Convert the Julian Day Number to the proleptic Gregorian Month, Day, Year."""
    L = jdn + 68569
    N = int(4 * L / 146_097)
    L = L - int((146097 * N + 3) / 4)
    I = int(4000 * (L + 1) / 1_461_001)
    L = L - int(1461 * I / 4) + 31
    J = int(80 * L / 2447)
    day = L - int(2447 * J / 80)
    L = int(J / 11)
    month = J + 2 - 12 * L
    year = 100 * (N - 49) + I + L
    return year, month, day

def julian_date_to_gregorian(jd):
    """Convert a decimal Julian Date to the equivalent proleptic Gregorian day and time."""
    jdn = int(jd)
    if jdn < 1_721_426:
        raise ValueError("Julian Day Numbers less than 1,721,426 are not supported, because"
                         "Python's date class cannot represent years before AD 1.")
    year, month, day = julian_day_number_to_gregorian(jdn)
    offset = timedelta(days=(jd % 1), hours=+12)
    dt = datetime.combine(date(year=year, month=month, day=day), time(0, 0, 0), tzinfo=timezone.utc)
    return dt + offset

jd_now = gregorian_to_julian_date(now)
print(julian_date_to_gregorian(jd_now), now)

2020-12-07 21:02:08.179953+00:00 2020-12-07 21:02:08.179961+00:00


We can confirm that this function gives back almost exactly the same time as originally specified. The difference is most likely due to floating point error in the microsecond calculation. Note that this function only gives dates in the proleptic Gregorian calendar, and does not account for the change from Julian to Gregorian calendar in 1582. This also means that the minimum Gregorian date is November 24, 4714 BC, corresponding to a Julian Day Number of 0.

In [4]:
print(julian_day_number_to_gregorian(0))

(-4713, 11, 24)


Note that since there is no Year 0, 4714 BC is the same as Year -4713.

## Simplified Formulas for Ephemerides

Now that we can calculate Julian Dates, we can use them to calculate ephemerides for the planets. The simplified formulas from JPL are due to E.M. Standish, and are available from the JPL website: <https://ssd.jpl.nasa.gov/?planet_pos>. The calculation procedure is further described in the accompanying PDF document: <https://ssd.jpl.nasa.gov/txt/aprx_pos_planets.pdf>

The simplified formulas give the classical Keplerian orbital elements ($h$, $e$, $M$, $\theta$, $\Omega$, and $\omega$) for each of the planets. Each element is given with its absolute value at a reference point in time, plus the rate of change of that element. The rates of change are all given _per century_, and the reference point is the [J2000.0 epoch](https://en.wikipedia.org/wiki/Epoch_(astronomy)#Julian_Dates_and_J2000) (January 1, 2000 at 12:00 PM (noon) UTC).

The first step is to determine the Julian Date given the Gregorian date and time, with the procedure described in the last section. With that completed, we will need to compute the number of Julian centuries that have elapsed since J2000.0 until the target date. We can do that by:

$$T = \frac{\mathrm{JDT} - 2451545}{36525}$$

where $\mathrm{JDT}$ is the desired Julian Date. Note that this should be floating point division, not integer division.

Next, we need to inspect the data. There are two tables given, one that is valid from AD 1800 to AD 1950, and the other that is valid from 3000 BC to AD 3000. The longer time range has a larger error relative to historical data. The tables here are reproduced from the JPL website: <https://ssd.jpl.nasa.gov/?planet_pos>

```{note}
These data are to be used as described in the related document
titled "Keplerian Elements for Approximate Positions of the
Major Planets" by E.M. Standish (JPL/Caltech) available from
the JPL Solar System Dynamics web site (<https://ssd.jpl.nasa.gov/>).
```

```{table} Keplerian elements and their rates, with respect to the mean ecliptic and equinox of J2000.0, valid for the time-interval 1800 AD - 2050 AD. Reproduced from Standish <https://ssd.jpl.nasa.gov/txt/p_elem_t1.txt>. Note that the original source has data for all eight major planets plus Pluto. The subset here is for demonstration only.
:align: center

|         | $a$ [AU, AU/Cy] | $e$ [rad, rad/Cy] | $i$ [deg, deg/Cy] | $L$ [deg, deg/Cy] | $\varpi$ [deg, deg/Cy] | $\Omega$ [deg, deg/Cy] |
|---------|---------------|-----------------|-----------------|-----------------|------------------------|------------------------|
| Mercury | 0.38709927    | 0.20563593      | 7.00497902      | 252.250324      | 77.4577963             | 48.3307659             |
|         | 0.00000037    | 0.00001906      | -0.0059475      | 149472.674      | 0.16047689             | -0.1253408             |
```

There are 6 columns and two rows associated with each planet. The first row gives the absolute value of the orbital element at the J2000.0 epoch and the second row gives the rate of change of the orbital element. Cy stands for an astronomical century (100 Julian years), or 36,525 days, although this notation is [outdated](https://www.iau.org/publications/proceedings_rules/units/). The six columns are:

1. Semimajor axis of the orbit ($a$), in [Astronomical Units](https://en.wikipedia.org/wiki/Astronomical_unit), equal to 149,597,870,700 m
2. Eccentricity ($e$), in radians
3. Inclination ($i$), in degrees
4. Mean Longitude ($L$), in degrees
5. Longitude of perihelion ($\varpi$), in degrees
6. Longitude of the ascending node ($\Omega$), in degrees

The coordinates are all given relative to the mean ecliptic plane, and use the vernal equinox of the J2000.0 epoch as the direction of the $X$ axis, for the purpose of defining 0° longitude.

Once we know the desired time relative to the J2000.0 epoch, in astronomical centuries, the value of any of the parameters given in the table can be found by:

$$Q = Q_0 + \dot{Q} T$$

where $Q$ is any of the properties given in the table, $\dot{Q}$ is the rate of change of that property, and $Q_0$ is the initial value at J2000.0. The next steps convert the mean longitude and longitude of the perihelion to the more familar true anomaly and argument of perihelion, respectively.

First, we will calculate the true anomaly using the mean longitude and longitude of perihelion. The mean longitude is related to the mean anomaly by:

$$M_e = L - \varpi$$

The mean anomaly must be in the range of 0-2$\pi$ radians, so it may be necessary to take the modulus of the result of this equation with 2$\pi$ to do that. Once the mean anomaly is known, we can use it and the eccentricity to solve Kepler's equation for the eccentric anomaly, $E$, using Newton iteration:

$$M_e = E - e \sin E$$

The eccentric anomaly can then be used to calculate the true anomaly:

$$\theta = 2 \tan^{-1}\left(\sqrt\frac{1 + e}{1 - e}\tan\frac{E}{2}\right)$$

Finally, we can calculate the argument of perihelion from the longitude of perihelion and the longitude of the ascending node:

$$\omega = \varpi - \Omega$$

We now have $a$, $e$, $i$, $\Omega$, $\omega$, and $\theta$, from which we can also calculate $h$ if necessary. In addition, we can convert the classical orbital elements to a state vector $\vector{r}$ and $\vector{v}$ for the planet, if needed.

The data shown in Table 1 are valid from AD 1800-AD 2050. If the data for 3000 BC-AD 3000 are used instead, there are a few extra terms, given in Table 2 below, which must be used in the equation for the mean anomaly, for the planets from Jupiter to Pluto.

```{table} Additional terms which must be added to the computation of $M_e$ for Jupiter through Pluto, 3000 BC to AD 3000, as described in the related document. Reproduced from Standish <https://ssd.jpl.nasa.gov/txt/p_elem_t2.txt>. Note that the original source has data for all the major planets from Jupiter to Neptune, plus Pluto. The subset here is for demonstration only.
:align: center

|            |     $b$       |    $c$       |    $s$     |     $f$      |
|------------|---------------|--------------|------------|--------------|
|Jupiter     | -0.00012452   | 0.06064060   | -0.35635438|   38.35125000|
```

The modified equation for the mean anomaly is:

$$M_e = L - \varpi + b T^2 + c\cos\left(f T\right) + s\sin\left(f T\right)$$

where $b$, $c$, $f$, and $s$ are from the table, and $T$ is the desired time. This is the only change necessary between the AD 1800-AD 2050 and the 3000 BC-AD 3000 data.

Let's calculate the orbital elements for Mercury on December 8, 2020 at 4:30 PM Eastern US time.

In [14]:
from scipy.optimize import newton
import numpy as np
mercury = {
    "a": 0.38709927, "a_dot": 0.00000037,
    "e": 0.20563593, "e_dot": 0.00001906,
    "i": 7.00497902, "i_dot": -0.0059475,
    "L": 252.250324, "L_dot": 149472.674,
    "long_peri": 77.4577963, "long_peri_dot": 0.16047689,
    "long_node":48.3307659, "long_node_dot": -0.1253408,
}
T_eph = datetime(month=12, day=8, year=2020, hour=21, minute=30, tzinfo=timezone.utc)
JDT = gregorian_to_julian_date(T_eph)
T = (JDT - 2_451_545) / 36_525

a = (mercury["a"] + mercury["a_dot"] * T) * 149_597_870.7
e = mercury["e"] + mercury["e_dot"] * T
i = np.radians(mercury["i"] + mercury["i_dot"] * T)
L = np.radians(mercury["L"] + mercury["L_dot"] * T)
long_peri = np.radians(mercury["long_peri"] + mercury["long_peri_dot"] * T)
long_node = np.radians(mercury["long_node"] + mercury["long_node_dot"] * T)

M_e = (L - long_peri) % (2 * np.pi)
def kepler(E, M_e, e):
    """Kepler's equation, to be used in a Newton solver."""
    return E - e * np.sin(E) - M_e

def d_kepler_d_E(E, M_e, e):
    """The derivative of Kepler's equation, to be used in a Newton solver.
    
    Note that the argument M_e is unused, but must be present so the function
    arguments are consistent with the kepler function.
    """
    return 1 - e * np.cos(E)

E = newton(func=kepler, fprime=d_kepler_d_E, x0=np.pi, args=(M_e, e))
theta = (2 * np.arctan(np.sqrt((1 + e) / (1 - e)) * np.tan(E / 2))) % (2 * np.pi)

omega = long_peri - long_node

print(f"𝑎 = {a:.5G} km", f"𝑒 = {e:.5F}", f"𝑖 = {np.degrees(i):.2F}°", f"𝜃 = {np.degrees(theta):.2F}°",
      f"𝜔 = {np.degrees(omega):.2F}°", f"𝛺 = {np.degrees(long_node):.2F}°", sep="\n")

𝑎 = 5.7909E+07 km
𝑒 = 0.20564
𝑖 = 7.00°
𝜃 = 159.92608753986138°
𝜔 = 29.186873275060883°
𝛺 = 48.30452275936573°


## Using a Library for Ephemerides

The formulas given in the last section are only approximate, with errors up to a few thousand kilometers, depending on the time interval and the particular planet of interest. JPL also produces much higher accuracy ephemerides (errors on the order of centimeters to a few kilometers) which are included in some specialized software. One example of such software available for Python is called [poliastro](https://docs.poliastro.space/en/stable/index.html), which we'll be using here. There are several other options as well:

* [pyEphem](https://rhodesmill.org/pyephem/) (outdated and replaced by the next item)
* [Skyfield](https://rhodesmill.org/skyfield/)
* [jplephem](https://pypi.org/project/jplephem/)
* [Astropy](https://www.astropy.org/) (also used by poliastro for some computations)
* [SpiceyPy](https://spiceypy.readthedocs.io/en/main/)

All these libraries use either the [NASA SPICE database](https://naif.jpl.nasa.gov/naif/data.html), in the form of SPK files, or the [NASA/JPL HORIZONS database](https://ssd.jpl.nasa.gov/?horizons), which we'll also discuss in the next section.

poliastro has a [number of methods](https://docs.poliastro.space/en/stable/user_guide.html#where-are-the-planets-computing-ephemerides) to produce ephemerides for the planets and also has a convenient interface to retrieve the classical orbtial parameters, which is why I've selected it here. The two main methods are:

* Lower-accuracy Astropy ephemerides, which rely on the [PyERFA](https://github.com/liberfa/pyerfa) library
* Higher-accuracy JPL ephemerides using the SPICE files

The Astropy ephemerids are included when poliastro is installed, while the JPL ephemerides require a separate download. We'll use the JPL ephemerides in this example. First, we need to determine a time at which the ephemerides should be calculated. poliastro requires an instance of the Astropy [`time.Time()`](https://docs.astropy.org/en/stable/time/) class to work, but we can create that using the `datetime` we set in the last section.

In [15]:
from astropy import time
epoch = time.Time(T_eph)

Next, we will set Astropy to use the JPL ephemerides.

In [16]:
from astropy.coordinates import solar_system_ephemeris
solar_system_ephemeris.set("jpl")

<ScienceState solar_system_ephemeris: 'jpl'>

In [21]:
from poliastro.ephem import Ephem
from poliastro.bodies import Sun, Mercury
from poliastro.twobody import Orbit
merc_ephem = Ephem.from_body(Mercury, epoch.tdb)
merc_orbit = Orbit.from_ephem(Sun, merc_ephem, epoch)
print(f"𝑎 = {merc_orbit.a:.5G}", f"𝑒 = {merc_orbit.ecc:.5F}", f"𝑖 = {merc_orbit.inc.to('degree'):.2F}",
      f"𝜃 = {merc_orbit.nu.to('degree'):.2F}", f"𝜔 = {merc_orbit.argp.to('degree'):.2F}",
      f"𝛺 = {merc_orbit.raan.to('degree')}", sep="\n")

𝑎 = 5.7567E+07 km
𝑒 = 0.20149
𝑖 = 28.64 deg
𝜃 = 164.05 deg
𝜔 = 62.56 deg
𝛺 = 10.790248210752672 deg


The parameters of the orbit are listed in the poliastro documentation: <https://docs.poliastro.space/en/stable/api/safe/twobody/twobody_index.html#poliastro.twobody.orbit.Orbit>.

Notice that the inclination is significantly different from the inclination we calculated previously. This is due to the _plane_ of the orbit that we defined. By default, poliastro uses the Earth-equatorial reference frame, which is inclined relative to the ecliptic plane.

In [22]:
print(merc_orbit.plane)

Planes.EARTH_EQUATOR


To change the plane to the ecliptic, we must change the plane when we create the ephemerides:

In [24]:
from poliastro.frames import Planes

merc_ephem = Ephem.from_body(Mercury, epoch.tdb, plane=Planes.EARTH_ECLIPTIC)
merc_orbit = Orbit.from_ephem(Sun, merc_ephem, epoch)
print(merc_orbit.plane, f"𝑖 = {merc_orbit.inc.to('degree'):.2F}", sep="\n")

Planes.EARTH_ECLIPTIC
𝑖 = 7.01 deg


As you can see, we now have the Earth ecliptic as the reference plane, and the inclination matches the previous calculation. The other major difference is in the true anomaly. This is due to the default choice of the reference point as the solar system barycenter. We can adjust the default reference point, which again we must do when we create the ephemerides. We do this by setting the `attractor` parameter for the ephemerides:

In [26]:
merc_ephem = Ephem.from_body(Mercury, epoch.tdb, attractor=Sun, plane=Planes.EARTH_ECLIPTIC)
merc_orbit = Orbit.from_ephem(Sun, merc_ephem, epoch)
print(f"𝜃 = {merc_orbit.nu.to('degree'):.2F}")

𝜃 = 159.87 deg


Now the result is much closer to the one we calculated previously.

Matlab has a similar function, called [`planetEphemeris`](https://www.mathworks.com/help/aerotbx/ug/planetephemeris.html). You have to install the Aerospace Toolbox, for which instructions can be found on the [Mathworks website](https://www.mathworks.com/matlabcentral/answers/101885-how-do-i-install-additional-toolboxes-into-an-existing-installation-of-matlab) or you can click the "Add-Ons" button in the Matlab ribbon toolbar and search for "aerospace toolbox". The first time you run `planetEphemeris`, it will also prompt you to install the SPK files to do the calculations.

With that all installed, `planetEphemeris` takes 3 arguments in the basic form:

1. A Julian Date
2. The center of the desired coordinate system
3. The target object

`planetEphemeris` then returns the position and velocity vector with respect to the center that you set. The default units are km and km/s.

```matlab
t1 = datetime('2020-December-08 21:30');
t1.TimeZone = 'UTC';
jd1 = juliandate(t1);
[position, velocity] = planetEphemeris(jd1, 'Sun','Mercury','432t');
angular_momentum = cross(position, velocity);
h = norm(angular_momentum);
i = acos(angular_momentum(end) / h);
rad2deg(i)

ans =

   28.5532

```

Once the `position` and `velocity` are obtained, we can use the methods from [Chapter 4](../chapter-4/4-orbital-elements-and-the-state-vector.md) to convert to the classical orbital elements. Note that the inclination is about 28 degrees, meaning that the default frame is the Earth-equatorial frame, equivalent to the [ICRF](https://en.wikipedia.org/wiki/International_Celestial_Reference_Frame). We can convert from this frame to the [ecliptic frame](https://en.wikipedia.org/wiki/Ecliptic_coordinate_system) by a rotation around the vernal equinox:

$$\mat{Q} = \begin{bmatrix}1 & 0 & 0\\0 & \cos\varepsilon & \sin\varepsilon\\0 & -\sin\varepsilon & \cos\varepsilon\end{bmatrix}$$

where $\varepsilon$ is the [_obliquity of the ecliptic_](https://en.wikipedia.org/wiki/Axial_tilt). The current value of $\varepsilon$ is 23.43657°. Thus, the transformation is accomplished by:

```matlab
obliquity = deg2rad(23.43657);
transform = [1 0 0; 0 cos(obliquity) sin(obliquity); 0 -sin(obliquity) cos(obliquity)];
pos = transform * position';
vel = transform * velocity';
am = cross(pos, vel);
h2 = norm(am);
rad2deg(acos(am(end) / h2))

ans =

    7.0055
```

With this coordinate transformation, we obtain the same inclination angle as before.

## Using JPL HORIZONS Web Interface

JPL also maintains a web browser based interface to the HORIZONS database. This interface can be used to determine all the same information as the previous methods, with a similar level of accuracy as the SPICE-focused methods in the previous section. The web interface is located at:

<https://ssd.jpl.nasa.gov/horizons.cgi>

The default interface of HORIZONS is shown in {numref}`Fig. {number} <horizons-default>`

:::{figure} horizons-default
<img src="../images/horizons-default.png" alt="Default interface for HORIZONS" width="50%">

The default web interface for HORIZONS.
:::

Each of the options can be changed by clicking the "Change" links. For our purposes, we can change the following:

1. _Ephemeris Type_: Either _Vector Table_ or _Orbital Elements_ is suitable, although the latter is more direct for this example
2. _Target Body_: On this option, there is a button to retrieve a list of the Sun and planets. We will choose that, and then choose _Mercury_.
3. _Center_: Choosing _Mercury_ in the previous step automatically sets the coordinate system center to the Sun center, but other options may be suitable here depending on your objective
4. _Time Span_: This can be used to generate a range of dates, or to input specific dates. We will choose _Switch to discrete time form_ for this example, and then input the date of interest, in UTC: 2020-12-08 21:30
5. _Table Settings_: Here, we want to change the units to _km & km/s_. Another useful option is the _reference plane_. The default of _ecliptic & mean equinox_ is appropriate for this example. You may also want to set the CSV output option, depending on how you will use the data.
6. _Display/Output_: On this page, you can choose the output format of your data. We will use the formatted HTML output in this example

When you've set the options for this example, the screen should appear as in {numref}`Fig. {number} <horizons-mercury>`.

:::{figure} horizons-mercury
<img src="../images/horizons-mercury.png" alt="Settings for Mercury ephemeris in HORIZONS" width="50%">
                                                                                                       
The settings used in this example for the ephemerides of Mercury.
:::

After clicking _Generate Ephemeris_, the output looks like {numref}`Fig. {number} <mercury-output>`.

:::{figure} mercury-output
<img src="../images/mercury-output.png" alt="Output for Mercury ephemeris from HORIZONS" width="75%">

A subset of the output from HORIZONS for Mercury showing the orbital elements.
:::

The HORIZONS output includes data about Mercury itself, the dates for which the ephemeris were calculated, and as shown in {numref}`Fig. {number} <mercury-output>`, the orbital elements of interest. Right below the orbital elements output is an explanation of what the acronyms mean. `IN` stands for inclination and `TA` is the true anomaly. Both of these elements match our previous calculated results for the date given.